In [1]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import pandas as pd
from collections import Counter

In [2]:
from sklearn.neighbors import KNeighborsClassifier

In [3]:
nba_players = pd.read_csv('./NBA_clean.csv', index_col=0)
nba_players = nba_players.sample(frac=1) #shuffle 100% of rows

#nba_players = nba_players.drop(['MIN', 'Age', 'Weight', 'BMI'], axis=1)
for stat in nba_players:
    print(stat)
nba_players = nba_players.reset_index(drop=True)
#nba_players

print(set(nba_players.loc[:, 'Pos']))
Counter(nba_players.loc[:, 'Pos'])

MIN
PTS
FGM
FGA
FG%
3PM
3PA
3P%
FTM
FTA
FT%
OREB
DREB
REB
AST
STL
BLK
TOV
PF
EFF
AST/TOV
STL/TOV
Age
Height
Pos
Weight
BMI
{'PG', 'C', 'PF', 'SG', 'SF'}


Counter({'C': 72, 'SG': 100, 'PG': 84, 'PF': 90, 'SF': 76})

In [4]:
def normalize(nba_players):
    for stat in nba_players.loc[:, nba_players.columns != 'Pos']:
        nba_players[stat] = nba_players[stat] - np.mean(nba_players[stat])
    for stat in nba_players.loc[:, nba_players.columns != 'Pos']:
        nba_players[stat] = nba_players[stat]/np.std(nba_players[stat])
    return nba_players

In [5]:
validation_data  = nba_players.loc[360:421, nba_players.columns != 'Pos']
validation_labels = nba_players.loc[360:421, 'Pos']

In [ ]:
splits = [0, 60 - 1, 120 - 1, 180 - 1, 240 - 1, 300 - 1, 360 - 1]

perf = []
klist = [2,3,4,5,6,7,8,9,10]
for k in klist:
    KNN = KNeighborsClassifier(n_neighbors=k)
    p = []
    for i in range(len(splits) - 1):
        #print(i)
        s1 = training_data = nba_players.loc[0:splits[i]-1, nba_players.columns != 'Pos']
        s2 = training_data = nba_players.loc[splits[i+1]+1:359, nba_players.columns != 'Pos']
        l1 = training_data = nba_players.loc[0:splits[i]-1, 'Pos']
        l2 = training_data = nba_players.loc[splits[i+1]+1:359,  'Pos']
        training_data = pd.concat([s1, s2])
        training_labels = pd.concat([l1, l2])
        testing_data = nba_players.loc[splits[i]:splits[i+1], nba_players.columns != 'Pos']
        testing_labels = nba_players.loc[splits[i]:splits[i+1], 'Pos']
        training_data = training_data.reset_index(drop=True)
        testing_data = testing_data.reset_index(drop=True)

        testing_data = normalize(testing_data)
        training_data = normalize(training_data)
        
        KNN.fit(training_data, training_labels)
        acc = KNN.score(testing_data, testing_labels)
        p.append(acc)
    perf.append(np.mean(p))
print(perf)
#KNN3 = KNeighborsClassifier(n_neighbors=3)
#KNN3.fit(nba_players.loc[0:359, nba_players.columns != 'Pos'], nba_players.loc[0:359, 'Pos'])

In [ ]:
#validate
print(klist[np.argmax(perf)])
KNN = KNeighborsClassifier(n_neighbors=klist[np.argmin(perf)])
training_data = nba_players.loc[0:359, nba_players.columns != 'Pos']
training_labels = nba_players.loc[0:359, 'Pos']
training_data = training_data.reset_index(drop=True)
training_data = normalize(training_data)
KNN.fit(training_data, training_labels)
validation_data = normalize(validation_data)
acc = KNN.score(validation_data, validation_labels)
print(acc)